In [1]:

import numpy as np
import json
import math

from imutils.video import VideoStream
import argparse
import datetime
import motmetrics as mm
import imutils
import time
from sklearn import preprocessing
import cv2 as cv
import time
from sort import *

import json
from utils import load_detections

from detection import Detection
from track import Track
from tracker import Tracker

acc = mm.MOTAccumulator(auto_id=True)

iou_overlaps = []
desc_dists = []
confusion_frames = []
confusion_tracks = []
confusion_distances =[]
colors = [[0,0,128],[0,255,0],[0,0,255],[255,0,0],[0,128,128],[128,0,128],[128,128,0],[255,255,0],[0,255,255],[255,255,0],[128,0,0],[0,128,0]
         ,[0,128,255],[0,255,128],[255,0,128],[128,255,0],[255,128,0],[128,255,255],[128,0,255],[128,128,128],[128,255,128]]
tracking_methods=['kalman_acc','kalman_vel']

#tracking_methods=['center_flow','keypoint_flow','kalman_center','kalman_corners','SORT']
detectors = ['yolo']
#detectors = ['ssd300','retinanet','yolo']
#'center_fow','keypoint_flow','kalman_center','kalman_corners',
datasets=['graal_1','graal_2','graal_3','graal_4']
times = {}
for dataset in datasets:
    times[dataset]={}
    images_input_path='../%s/'%dataset
    image_id_prefix= dataset
    frame_width=1032
    frame_height=778
    if(dataset=='venc'):
        frame_width = 1280
        frame_height = 960
    if(dataset=='modd'):
        frame_width=640
        frame_height=464
    if(dataset=='garda_1' or dataset=='garda_2'):
        frame_width=1280
        frame_height=720
    if(dataset=='mot_1'):
        frame_width=768
        frame_height=576
    iou_threshold = 0.1
    for detector in detectors:
        times[dataset][detector] = {}
        boat_class=8
        min_conf=0
        if(detector=='ssd300'):
            boat_class=4
            
        if(detector=='def'):
            boat_class=1
        

        path = '%s/%s_videos'%(detector,image_id_prefix)
        detections = load_detections(image_id_prefix,detector,boat_class,min_conf)
        
        for tracking_method in tracking_methods:
            times[dataset][detector][tracking_method] = []
            video_output_path='%s/%s(%.1f conf).avi'%(path,tracking_method,min_conf)
            json_output_path='%s/%s(%.1f conf).json'%(path,tracking_method,min_conf)
            out_tracking = cv.VideoWriter(video_output_path,cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
            frameCount =0
            no_tracking_res = [] 
            tracking_res = []
            kalman_trackers=[]
            # initialize the first frame in the video stream
            frameCount =0
            step_up = 0.1
            step_down = 0.2
            print('Running: Dataset:%s, Detector:%s, Tracker:%s, @%dx%d, File Name %s'%(dataset,detector,tracking_method,frame_width,frame_height,video_output_path))
            preds = []
            tracks=[]
            started = False
            multiplier=0
            cc=0
            prev_frame=None
            
            total_frames=900
            if(tracking_method=='SORT'):
                mot_tracker = Sort()
            else:
                tracker_wrapper = Tracker(tracking_method)
                tracker_wrapper.frame_width = frame_width
                tracker_wrapper.frame_height = frame_height
                if(dataset=='modd'):
                    tracker_wrapper.A = np.array([int(frame_width/2),int(frame_height/2)])
                    tracker_wrapper.B = np.array([int(frame_width/5),int(frame_height-1)])
                    tracker_wrapper.C= np.array([int(4*frame_width/5),frame_height-1])
                elif(dataset=='graal_1' or dataset == 'graal_2' or dataset=='garda_1' or dataset=='garda_2'):
                    tracker_wrapper.A = np.array([int(frame_width/2),int(frame_height/2)])
                    tracker_wrapper.B = np.array([int(frame_width/5),int(frame_height-1)])
                    tracker_wrapper.C= np.array([int(4*frame_width/5),frame_height-1])
            while frameCount<total_frames:
                
                # grab the current frame and initialize the occupied/unoccupied
                # text
                frame = cv.imread('%s%s.jpg'%(images_input_path,str(frameCount+1).zfill(5)))
                
                
                    
                
                
                # if the frame could not be grabbed, then we have reached the end
                # of the video
                if frame is None:
                    break

                if(frameCount<0):
                    continue

                preds = []
                
                start= time.time()
                
                
                if '%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5)) in detections:
                    preds = detections['%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5))]
                
                for det in preds:
                    det.calc_hog_descriptor(frame)
                    
                
                tracker_wrapper.track(preds,frame,prev_frame)


                to_display = tracker_wrapper.get_display_tracks()





                    #print(acc.mot_events.loc[frameId])
               
                col_points = []#tracker_wrapper.get_collision_points()
                if(len(col_points)>0):
                    col = [0,0,255]
                    cv.putText(frame,'Collision detected!', (20, 20),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    
                
                else:
                    col = [0,255,255]
                
                
               
                
                for p in col_points:
                    cv.circle(frame,(int(p[0]),int(p[1])),8,(0,0,255),1)
                i=0
                for box in to_display:
                    
                # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
                   
                    xmin = int(box.xmin)
                    ymin = int(box.ymin)
                    xmax =int(box.xmax)
                    ymax =int(box.ymax)
                    
                    cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), colors[int(box.track_id)%len(colors)], 3)
                    #cv.rectangle(frame, (int(box.new_box[0] - box.new_box[2]/2), int(box.new_box[1] - box.new_box[3]/2)), (int(box.new_box[0] + box.new_box[2]/2),int(box.new_box[1] + box.new_box[3]/2)), (255,255,255), 2)
                    p1 = box.center()
                    p2 = box.center()
                    p2[0] += box.offset[0]*3
                    p2[1] += box.offset[1]*100
                    
                    #cv.arrowedLine(frame,(int(p1[0]),int(p1[1])),(int(p2[0]),int(p2[1])),(0,0,255),2)
                    if(tracking_method=='kalman_center'):
                        
                        cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                    #if(tracking_method=='kalman_corners'):
                        #cv.circle(frame,(int(predictions[i][0][0]),int(predictions[i][1][0])),5,(255,0,0),2)
                        #cv.circle(frame,(int(predictions[i][2][0]),int(predictions[i][3][0])),5,(255,0,0),2)
                    cv.putText(frame,'{:.2f}'.format( box.conf), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    

                    tracking_res.append({"image_id" : frameCount+1, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)],"id":box.track_id, "score" :1})# np.minimum(1.0,np.maximum(box.conf,0.5))})
                    #f.write("graal_2/%s.jpg,%s,%d,%f,%f,%f,%f,%f\n"%(str(frameCount+1).zfill(5),classes[int(box[1])],box[1],box[2],xmin,ymin,xmax,ymax))
                    i+=1
                    times[dataset][detector][tracking_method].append(time.time()-start)
                
                out_tracking.write(frame)
                #cv.imwrite('debug_frames/%s.jpg'%str(frameCount+1),frame)
                frameCount+=1
                prev_frame=frame
                flow = None
            # cleanup the camera and close any open windows

            out_tracking.release()
            
            
    

            with open(json_output_path, 'w') as outfile:  
                json.dump(tracking_res, outfile)
                
for k in times.keys():
    print(k)
    yolo = times[k]['yolo']
    for tr in yolo.keys():
        print(tr,np.average(yolo[tr]))
        

Running: Dataset:graal_1, Detector:yolo, Tracker:kalman_acc, @1032x778, File Name yolo/graal_1_videos/kalman_acc(0.0 conf).avi


C:\Users\issam\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


Running: Dataset:graal_1, Detector:yolo, Tracker:kalman_vel, @1032x778, File Name yolo/graal_1_videos/kalman_vel(0.0 conf).avi
Running: Dataset:graal_2, Detector:yolo, Tracker:kalman_acc, @1032x778, File Name yolo/graal_2_videos/kalman_acc(0.0 conf).avi
Running: Dataset:graal_2, Detector:yolo, Tracker:kalman_vel, @1032x778, File Name yolo/graal_2_videos/kalman_vel(0.0 conf).avi
Running: Dataset:graal_3, Detector:yolo, Tracker:kalman_acc, @1032x778, File Name yolo/graal_3_videos/kalman_acc(0.0 conf).avi
Running: Dataset:graal_3, Detector:yolo, Tracker:kalman_vel, @1032x778, File Name yolo/graal_3_videos/kalman_vel(0.0 conf).avi
Running: Dataset:graal_4, Detector:yolo, Tracker:kalman_acc, @1032x778, File Name yolo/graal_4_videos/kalman_acc(0.0 conf).avi
Running: Dataset:graal_4, Detector:yolo, Tracker:kalman_vel, @1032x778, File Name yolo/graal_4_videos/kalman_vel(0.0 conf).avi
graal_1
kalman_acc 0.016713756653997634
kalman_vel 0.016737702289058586
graal_2
kalman_acc 0.033231061161234136

In [ ]:
print(min_conf)

In [ ]:
cv.KalmanFilter??

In [12]:

import numpy as np
import json
import math

from imutils.video import VideoStream
import argparse
import datetime
import motmetrics as mm
import imutils
import time
from sklearn import preprocessing
import cv2 as cv
import time
from sort import *

import json
from utils import load_detections

from detection import Detection
from track import Track
from tracker import Tracker

acc = mm.MOTAccumulator(auto_id=True)
total_frames=900
iou_overlaps = []
desc_dists = []
confusion_frames = []
confusion_tracks = []
confusion_distances =[]
colors = [[0,0,128],[0,255,0],[0,0,255],[255,0,0],[0,128,128],[128,0,128],[128,128,0],[255,255,0],[0,255,255],[255,255,0],[128,0,0],[0,128,0]
         ,[0,128,255],[0,255,128],[255,0,128],[128,255,0],[255,128,0],[128,255,255],[128,0,255],[128,128,128],[128,255,128]]
tracking_methods=['SORT']
#tracking_methods=['center_flow','keypoint_flow','kalman_center','kalman_corners','SORT']
detectors = ['ssd300']
#detectors = ['ssd300','retinanet','yolo']
#'center_fow','keypoint_flow','kalman_center','kalman_corners',
datasets=['graal_4']
times = {}
for dataset in datasets:
    times[dataset]={}
    images_input_path='../%s/'%dataset
    image_id_prefix= dataset
    frame_width=1032
    frame_height=778
    if(dataset=='venc'):
        frame_width = 1280
        frame_height = 960
    if(dataset=='modd'):
        frame_width=640
        frame_height=464
    if(dataset=='garda_1' or dataset=='garda_2'):
        frame_width=1280
        frame_height=720
    if(dataset=='mot_1'):
        frame_width=768
        frame_height=576
    iou_threshold = 0.1
    for detector in detectors:
        times[dataset][detector] = {}
        boat_class=8
        min_conf=0
        if(detector=='ssd300'):
            boat_class=4
            min_conf=0
        if(detector=='def'):
            boat_class=1
        

        path = '%s/%s_videos'%(detector,image_id_prefix)
        detections = load_detections(image_id_prefix,detector,boat_class,0.5)
        
        for tracking_method in tracking_methods:
            times[dataset][detector][tracking_method] = []
            video_output_path='%s/%s.avi'%(path,tracking_method)
            json_output_path='%s/%s.json'%(path,tracking_method)
            out_tracking = cv.VideoWriter(video_output_path,cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
            frameCount =0
            
            # initialize the first frame in the video stream
            
            
            print('Running: Dataset:%s, Detector:%s, Tracker:%s, @%dx%d'%(dataset,detector,tracking_method,frame_width,frame_height))
            preds = []
            if(tracking_method=='SORT'):
                mot_tracker = Sort()
            tracking_res = []
            while frameCount<total_frames:
                
                # grab the current frame and initialize the occupied/unoccupied
                # text
                frame = cv.imread('%s%s.jpg'%(images_input_path,str(frameCount+1).zfill(5)))
                
                
                    
                
                
                # if the frame could not be grabbed, then we have reached the end
                # of the video
                if frame is None:
                    break

                if(frameCount<0):
                    continue

                preds = []
                if '%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5)) in detections:

                    for box in detections['%s/%s.jpg'%(image_id_prefix,str(frameCount+1).zfill(5))]:

                        if(box.conf<min_conf):

                            continue
                        
                        

                        temp_pred =  np.array([box.xmin,box.ymin,box.xmax,box.ymax,box.conf])

                        preds.append(temp_pred)
                        
                          
                
                start= time.time()
                if(tracking_method=='SORT'):
                    preds = np.asarray(preds)
                    trackers = mot_tracker.update(preds)
                    to_display = []
                    for itrk,tracker in enumerate(trackers):
                        to_display.append(np.array([tracker[4],tracker[0],tracker[1],tracker[2],tracker[3]]))
               
                i=0
                for box in to_display:
                    
                # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
                   
                    xmin = int(box[1])
                    ymin = int(box[2])
                    xmax =int(box[3])
                    ymax =int(box[4])
                    
                    cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), colors[int(box[0])%len(colors)], 3)
                    

                    tracking_res.append({"image_id" : frameCount+1, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : 1.0,"id":int(box[0])})
                    i+=1
                    times[dataset][detector][tracking_method].append(time.time()-start)
                
                out_tracking.write(frame)
                #cv.imwrite('debug_frames/%s.jpg'%str(frameCount+1),frame)
                frameCount+=1
                prev_frame=frame
                flow = None
            # cleanup the camera and close any open windows

            out_tracking.release()
            
            


            with open(json_output_path, 'w') as outfile:  
                json.dump(tracking_res, outfile)
                
for k in times.keys():
    print(k)
    yolo = times[k]['ssd300']
    for tr in yolo.keys():
        print(tr,np.average(yolo[tr]))

Running: Dataset:graal_4, Detector:ssd300, Tracker:SORT, @1032x778
graal_4
SORT 0.0009647131872721045
